In [ ]:
# tensorflow
import tensorflow as tf
from tensorflow import keras

# helpers
import pickle
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
plt.style.use('ggplot')

In [ ]:
##### tool functions #####

# plot an image in a subplot
def subplot_image(image, label, nrows=1, ncols=1, iplot=0):
    plt.subplot(nrows, ncols, iplot + 1)
    plt.imshow(image, cmap=plt.cm.binary)
    plt.xlabel(label)
    plt.xticks([])
    plt.yticks([])
    
# plot a bar chart in a subplot
def subplot_bar(data, true_label, nrows=1, ncols=1, iplot=0):
    plt.subplot(nrows, ncols, iplot + 1)
    chart = plt.bar(np.arange(len(data)), data, color='gray')
    predicted_label = np.argmax(data)
    chart[predicted_label].set_color('red')
    chart[true_label].set_color('green')
    plt.xticks(np.arange(len(data)))
    plt.yticks([])
    plt.ylim([0, 1])
    plt.gca().set_aspect(len(data))
    plt.title('Probability', fontsize=12)
    

# 0. Dataset: Fashion-MNIST
![fashion.png](https://i.ibb.co/N7wPnqs/fashion.png)

In [ ]:
# load data
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# normalise images
train_images = train_images / 255.0
test_images = test_images / 255.0

# string labels
string_labels = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat',
                 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# info
print("number of training data: %d" % len(train_labels))
print("number of test data: %d" % len(test_labels))
print("image pixels: %s" % str(train_images[0].shape))

In [ ]:
# plot data
nrows = 4
ncols = 8
plt.figure(dpi=100, figsize=(ncols * 2, nrows * 2.2))
for i in np.arange(nrows * ncols):
    title = "%d: %s" % (train_labels[i], string_labels[train_labels[i]])
    subplot_image(train_images[i], title, nrows, ncols, i)
plt.show()

---

# 1. Supervised: DNN

We start with a simple DNN that has only one hidden layer. This is a classification problem, an instance of supervised learning.

### Create the DNN
Our first DNN looks like this:
![dense.jpeg](https://i.ibb.co/W0dDDfm/dense-001.jpg)

#### Concepts to learn:
* Input and output layers
* Hidden layers
* Dense layers
* Activation functionn
* Dropout and overfitting

In [ ]:
# define model
dnn = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
], name='DNN_01')

# print summary
dnn.summary()

# save initial weights
dnn_initial_weights = dnn.get_weights().copy()

### Compile the DNN
#### Concepts to learn:
* Optimizer
* Loss function
* Metrics

In [ ]:
# compile model
dnn.compile(optimizer='adam',
            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

### Train the DNN
#### Concepts to learn:
* Epoch
* Batch size

In [ ]:
# to save time, you can load the trained model instead of doing the training
# this training takes about 1 minute (epochs=20, batch_size=32)
just_load_trained = True

if just_load_trained:
    # load model
    dnn.load_weights('trained_models/dnn01/trained.h5')
    # load training history
    with open('trained_models/dnn01/history.bin', 'rb') as fin:
        training_history = pickle.load(fin)
else:
    # reset to initial weights
    dnn.set_weights(dnn_initial_weights)
    # train model
    training_history = dnn.fit(train_images, train_labels, epochs=20, batch_size=32)
    training_history = training_history.history
    # save model
    dnn.save_weights('trained_models/dnn01/trained.h5')
    # save training history
    with open('trained_models/dnn01/history.bin', 'wb') as fout:
        pickle.dump(training_history, fout)
        
# plot training history
plt.figure(dpi=100)
plt.plot(training_history['loss'], label='Loss')
plt.plot(training_history['accuracy'], label='Accuracy')
plt.xticks(np.arange(0, len(training_history['loss']) + 1, 2))
plt.xlabel('Epoch')
plt.ylabel('Loss or accuracy')
plt.legend()
plt.show()

### Evaluate the DNN with test data

In [ ]:
# evaluate model
test_loss, test_acc = dnn.evaluate(test_images, test_labels)
print('\nTest loss:', test_loss)
print('\nTest accuracy:', test_acc)

### Make predictions

In [ ]:
# append a softmax layer to normalise the output
probability_model = keras.Sequential([dnn, keras.layers.Softmax()])

# use test data to make predictions
test_probabilities = probability_model.predict(test_images)
test_pred_labels = np.argmax(test_probabilities, axis=1)

# plot the test images along with their predicted and true labels.
nrows = 5
ncols = 3
plt.figure(dpi=100, figsize=(6 * ncols, 2.5 * nrows))
np.random.seed(0)
for iplot, itest in enumerate(np.random.randint(0, len(test_labels), nrows * ncols)):
    # plot image
    title = "%d: %s" % (test_labels[itest], string_labels[test_labels[itest]])
    subplot_image(test_images[itest], title, nrows, ncols * 3, iplot * 3)
    # plot bar chart of probability
    subplot_bar(test_probabilities[itest], test_labels[itest], nrows, ncols * 3, iplot * 3 + 1)
    # additional texts
    subplot_image([[0]], '', nrows, ncols * 3, iplot * 3 + 2)
    correct = test_labels[itest] == test_pred_labels[itest]
    textcolor = 'g' if correct else 'r'
    plt.text(-.5, -.3, "Truth: %s" % string_labels[test_labels[itest]])
    plt.text(-.5, -.1, "Prediction: %s" % string_labels[test_pred_labels[itest]], c=textcolor)
    plt.text(-.5, .1, "Probability: %.0f%%" % (test_probabilities[itest].max() * 100), c=textcolor)
    plt.text(-.5, .3, "CORRECT" if correct else 'INCORRECT', c=textcolor, weight='bold')
plt.show()

---

# 2. Supervised: CNN

For the same classification problem, we now use a CNN like this:
![layer.jpeg](https://i.ibb.co/VDJ301m/layer.jpg)

#### Concepts to learn:
* Convolutional layers
* Max pooling layers

In [ ]:
# create model
cnn = keras.models.Sequential(name='CNN')
# 1 input => convolution
cnn.add(keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(28, 28, 1)))
# 2 max pooling
cnn.add(keras.layers.MaxPooling2D((2, 2)))
# 3 convolution
cnn.add(keras.layers.Conv2D(32, (5, 5), activation='relu'))
# 4 max pooling
cnn.add(keras.layers.MaxPooling2D((2, 2)))
# 5 flatten
cnn.add(keras.layers.Flatten())
# 6 dense
cnn.add(keras.layers.Dense(64, activation='relu'))
# 7 dense => output
cnn.add(keras.layers.Dense(10))

# print summary
cnn.summary()

# save initial weights
cnn_initial_weights = cnn.get_weights().copy()

# optimizer, loss, metrics
cnn.compile(optimizer='adam',
            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

In [ ]:
# append channel dimension because we only have one channel (gray-scale)
def append_channel_dim(img):
    return np.array([img]).transpose([1, 2, 3, 0])

In [ ]:
# to save time, you can load the trained model instead of doing the training
# this training takes about 6 minutes (epochs=20, batch_size=32)
just_load_trained = True

if just_load_trained:
    # load model
    cnn.load_weights('trained_models/cnn01/trained.h5')
    # load training history
    with open('trained_models/cnn01/history.bin', 'rb') as fin:
        training_history = pickle.load(fin)
else:
    # reset to initial weights
    cnn.set_weights(cnn_initial_weights)
    # train model
    training_history = cnn.fit(append_channel_dim(train_images), train_labels, 
                               epochs=20, batch_size=32)
    training_history = training_history.history
    # save model
    cnn.save_weights('trained_models/cnn01/trained.h5')
    # save training history
    with open('trained_models/cnn01/history.bin', 'wb') as fout:
        pickle.dump(training_history, fout)
        
# plot training history
plt.figure(dpi=100)
plt.plot(training_history['loss'], label='Loss')
plt.plot(training_history['accuracy'], label='Accuracy')
plt.xticks(np.arange(0, len(training_history['loss']) + 1, 2))
plt.xlabel('Epoch')
plt.ylabel('Loss or accuracy')
plt.legend()
plt.show()

In [ ]:
# evaluate model
test_loss, test_acc = cnn.evaluate(append_channel_dim(test_images), test_labels, verbose=1)
print('\nTest loss:', test_loss)
print('\nTest accuracy:', test_acc)

In [ ]:
# append a softmax layer to normalise the output
probability_model = keras.Sequential([cnn, keras.layers.Softmax()])

# use test data to make predictions
test_probabilities = probability_model.predict(append_channel_dim(test_images))
test_pred_labels = np.argmax(test_probabilities, axis=1)

# plot the test images along with their predicted and true labels.
nrows = 5
ncols = 3
plt.figure(dpi=100, figsize=(6 * ncols, 2.5 * nrows))
np.random.seed(0)
for iplot, itest in enumerate(np.random.randint(0, len(test_labels), nrows * ncols)):
    # plot image
    title = "%d: %s" % (test_labels[itest], string_labels[test_labels[itest]])
    subplot_image(test_images[itest], title, nrows, ncols * 3, iplot * 3)
    # plot bar chart of probability
    subplot_bar(test_probabilities[itest], test_labels[itest], nrows, ncols * 3, iplot * 3 + 1)
    # additional texts
    subplot_image([[0]], '', nrows, ncols * 3, iplot * 3 + 2)
    correct = test_labels[itest] == test_pred_labels[itest]
    textcolor = 'g' if correct else 'r'
    plt.text(-.5, -.3, "Truth: %s" % string_labels[test_labels[itest]])
    plt.text(-.5, -.1, "Prediction: %s" % string_labels[test_pred_labels[itest]], c=textcolor)
    plt.text(-.5, .1, "Probability: %.0f%%" % (test_probabilities[itest].max() * 100), c=textcolor)
    plt.text(-.5, .3, "CORRECT" if correct else 'INCORRECT', c=textcolor, weight='bold')
plt.show()

---

# 3. Semi-supervised: Autoencoder

<strong> What is autoencoder? </strong>

An autoencoder is a type of artificial neural network used to learn efficient data codings in an unsupervised manner. The aim of an autoencoder is to learn a representation (encoding) for a set of data, typically for dimensionality reduction, by training the network to ignore signal “noise”. Along with the reduction side, a reconstructing side is learnt, where the autoencoder tries to generate from the reduced encoding a representation as close as possible to its original input, hence its name. -- from Wikipedia

![autoencoder.jpg](https://i.ibb.co/Zg2TNGP/Screenshot-2020-07-12-at-4-21-09-AM.png)

#### Concepts to learn:
* Autoencoder
* Encoder
* Decoder
* dimensionality reduction

### Target class for training
We train the autoencoder using images from one <strong> target class </strong>, such as "Sneaker". The trained autoencoder should be able to distinguish whether a test image belongs to the target class based on the reconstruction error: an image belonging to the target class tends to have a small reconstruction error.

In [ ]:
# choose a target class for training
# 'T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat',
# 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
target_class = 'Sneaker'
target_label = string_labels.index(target_class)

# size of the "bottleneck"
bottleneck = 8

In [ ]:
# get image subset of the chosen class
target_train_images = train_images[np.where(train_labels == target_label)[0]]
target_test_images = test_images[np.where(test_labels == target_label)[0]]
print('Total number of images labelled "%s":' % (target_class,))
print('%d in training data' % (target_train_images.shape[0],))
print('%d in test data' % (target_test_images.shape[0],))

### Build the autoencoder

In [ ]:
# create model
autoencoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256, activation='selu'),
    keras.layers.Dense(128, activation='selu'),
    keras.layers.Dense(bottleneck, activation='selu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(128, activation='selu'),
    keras.layers.Dense(256, activation='selu'),
    keras.layers.Dense(784, activation='sigmoid'),
    keras.layers.Reshape((28, 28))
], name='Autoencoder')

# print summary
autoencoder.summary()

# save initial weights
ae_initial_weights = autoencoder.get_weights().copy()

# compile
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# to save time, you can load the trained model instead of doing the training
# this training takes about 5 minutes (epochs=300, batch_size=32)
just_load_trained = True

if just_load_trained:
    # load model
    autoencoder.load_weights('trained_models/ae01/trained_%s_%s.h5' % (target_class, bottleneck))
    # load training history
    with open('trained_models/ae01/history_%s_%s.bin' % (target_class, bottleneck), 'rb') as fin:
        training_history = pickle.load(fin)
else:
    # reset to initial weights
    autoencoder.set_weights(ae_initial_weights)
    # train model
    training_history = autoencoder.fit(target_train_images, target_train_images,
                                       epochs=300, batch_size=32,
                                       validation_data=(target_test_images, target_test_images))
    training_history = training_history.history
    # save model
    autoencoder.save_weights('trained_models/ae01/trained_%s_%s.h5' % (target_class, bottleneck))
    # save training history
    with open('trained_models/ae01/history_%s_%s.bin' % (target_class, bottleneck), 'wb') as fout:
        pickle.dump(training_history, fout)
        
# plot training history
plt.figure(dpi=100)
plt.plot(training_history['loss'], label='Loss')
plt.xticks(np.arange(0, len(training_history['loss']) + 1, 50))
plt.xlabel('Epoch')
plt.ylabel('Loss or accuracy')
plt.legend()
plt.show()

### Reconstruct test images, evaluate loss and classify

A small loss (or reconstruction error) indicates a high probability that a test image belongs to the target class.

In [ ]:
# reconstruct training and test images
reconstructed_test_images = autoencoder.predict(test_images)

# specify a threshold loss
# an image belongs to the target class if loss is smaller than this threshold loss
threshold_loss = 0.3

In [ ]:
# plot original and reconstructed test images
nrows = 5
ncols = 3
plt.figure(dpi=100, figsize=(6 * ncols, 2.5 * nrows))
np.random.seed(0)
for iplot, itest in enumerate(np.random.randint(0, len(test_images), nrows * ncols)):
    # plot original image
    subplot_image(test_images[itest], 'origional', nrows, ncols * 3, iplot * 3)
    # plot reconstructed image
    subplot_image(reconstructed_test_images[itest], 'reconstructed', nrows, ncols * 3, iplot * 3 + 1)
    # additional texts
    subplot_image([[0]], '', nrows, ncols * 3, iplot * 3 + 2)
    # truth
    truth = (test_labels[itest] == target_label)
    # loss evaluation
    loss = autoencoder.evaluate(np.array([test_images[itest]]), 
                                np.array([test_images[itest]]), verbose=0)
    # prediction
    predict = (loss < threshold_loss)
    # correct or not?
    correct = (truth == predict)
    textcolor = 'g' if correct else 'r'
    plt.text(-.5, -.3, 'Truth: %s' % string_labels[test_labels[itest]])
    plt.text(-.5, -.1, 'Loss: %.2f' % loss, c=textcolor)
    plt.text(-.5, .2, 'Prediction:\n%s' % (target_class if predict else 'Non-%s' % target_class), c=textcolor)
    plt.text(-.5, .4, "CORRECT" if correct else 'INCORRECT', c=textcolor, weight='bold')
plt.show()

## Generate new synthetic images by changing the bottleneck features

A trained autoencoder can also be used to decompress an image (using the encoder) and to generate new synthetic images (using the decoder) after modifying the key features in the bottleneck layer.

### Extract the encoder and decoder (the first and second half of our autoencoder)

In [ ]:
# encoder
encoder = keras.models.Sequential([
    autoencoder.layers[0],
    autoencoder.layers[1],
    autoencoder.layers[2],
    autoencoder.layers[3]
], name='Encoder')

# decoder
decoder = keras.models.Sequential([
    keras.layers.Input(shape=(bottleneck,)),
    autoencoder.layers[-4],
    autoencoder.layers[-3],
    autoencoder.layers[-2],
    autoencoder.layers[-1]
], name='Decoder')

# print summary
encoder.summary()
print('\n')
decoder.summary()

### Compute the bottleneck using the encoder

In [ ]:
# encoding
target_train_images_encoded = encoder.predict(target_train_images)

# reconstruct training images
reconstructed_target_train_images = autoencoder.predict(target_train_images)

In [ ]:
# pick an image index here (0~5999)
image_index = 1000

# original bottleneck data
encoded_original = target_train_images_encoded[image_index]
print('Key features at the bottleneck:')
print(encoded_original)

### Generate new synthetic images using the decoder

In [ ]:
# modify the key features
encoded_modified = encoded_original.copy()
encoded_modified[0] = 2
encoded_modified[1] = 2

# decode modified
decoded_modified = decoder.predict(np.array([encoded_modified]))[0]

# plot images
plt.figure(dpi=100, figsize=(10, 3))
subplot_image(target_train_images[image_index], 'Original', 1, 3, 0)
subplot_image(reconstructed_target_train_images[image_index], 'Reconstructed original', 1, 3, 1)
subplot_image(decoded_modified, 'Reconstructed modified', 1, 3, 2)
plt.show()

In [ ]:
# use widgets for dynamic modification
import ipywidgets as widgets

In [ ]:
# create sliders
sliders = []
argdict = {}
for i in np.arange(bottleneck):
    sliders.append(widgets.FloatSlider(min=-5, max=5, step=.01, value=encoded_original[i], 
                                       description='Feature %d:' % (i + 1,),
                                       layout=widgets.Layout(width='auto', height='auto')))
    # associate sliders with varaibles passed to the respondent function
    argdict['v%d' % (i + 1)] = sliders[i]
ui = widgets.VBox(sliders)

# the respondent function
# TODO: make argument count consistent with bottleneck size
def respond(v1, v2, v3, v4, v5, v6, v7, v8):
    # decode
    encoded_modified = np.array([v1, v2, v3, v4, v5, v6, v7, v8])
    decoded_modified = decoder.predict(np.array([encoded_modified]))[0]
    # plot images
    plt.figure(dpi=100, figsize=(10, 3))
    subplot_image(target_train_images[image_index], 'Original', 1, 3, 0)
    subplot_image(reconstructed_target_train_images[image_index], 'Reconstructed original', 1, 3, 1)
    subplot_image(decoded_modified, 'Reconstructed modified', 1, 3, 2)
    plt.show()

# craete UI
out = widgets.interactive_output(respond, argdict)
display(ui, out)

# 4. Unsupervised: Autoencoder + K-means

In this example, we use K-means to cluster the images. Unsupervised clustering is challenged by a large input dimensionality (such as 28 x 28 here), so we first use an autoencoder to first reduce the input dimensionality and then perform clustering in the bottleneck feature space.

#### Concepts to learn:
* Clustering
* K-means

In [ ]:
# import K-means from sklearn
from sklearn.cluster import KMeans
import sklearn.metrics

### Build and train the autoencoder with all the 10 classes

In [ ]:
# size of the "bottleneck"
bottleneck_all = 8

# create model
ae_all = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256, activation='selu'),
    keras.layers.Dense(128, activation='selu'),
    keras.layers.Dense(bottleneck_all, activation='selu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(128, activation='selu'),
    keras.layers.Dense(256, activation='selu'),
    keras.layers.Dense(784, activation='sigmoid'),
    keras.layers.Reshape((28, 28))
], name='Autoencoder-All')

# print summary
ae_all.summary()

# save initial weights
ae_all_initial_weights = ae_all.get_weights().copy()

# compile
ae_all.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# to save time, you can load the trained model instead of doing the training
# this training takes about 9 minutes (epochs=100, batch_size=64)
just_load_trained = True

if just_load_trained:
    # load model
    ae_all.load_weights('trained_models/ae02/trained_%s.h5' % (bottleneck_all,))
    # load training history
    with open('trained_models/ae02/history_%s.bin' % (bottleneck_all,), 'rb') as fin:
        training_history = pickle.load(fin)
else:
    # reset to initial weights
    ae_all.set_weights(ae_all_initial_weights)
    # train model
    training_history = ae_all.fit(train_images, train_images, epochs=100, batch_size=64,
                                  validation_data=(test_images, test_images))
    training_history = training_history.history
    # save model
    ae_all.save_weights('trained_models/ae02/trained_%s.h5' % (bottleneck_all,))
    # save training history
    with open('trained_models/ae02/history_%s.bin' % (bottleneck_all,), 'wb') as fout:
        pickle.dump(training_history, fout)
        
# plot training history
plt.figure(dpi=100)
plt.plot(training_history['loss'], label='Loss')
plt.xticks(np.arange(0, len(training_history['loss']) + 1, 10))
plt.xlabel('Epoch')
plt.ylabel('Loss or accuracy')
plt.legend()
plt.show()

### Encode the images

In [ ]:
# extract encoder
encoder_all = keras.models.Sequential([
    ae_all.layers[0],
    ae_all.layers[1],
    ae_all.layers[2],
    ae_all.layers[3]
], name='Encoder-All')

# encode images
train_images_encoded = encoder_all.predict(train_images)

### Cluster the encoded images

In [ ]:
# build model
kmeans_model = KMeans(n_clusters=10, max_iter=300)

# train and predict
kmeans_labels = kmeans_model.fit_predict(train_images_encoded)

# print scores
print('Homogeneity score = %.3f' % sklearn.metrics.homogeneity_score(train_labels, kmeans_labels))
print('Completeness score = %.3f' % sklearn.metrics.completeness_score(train_labels, kmeans_labels))
print('V-measure score = %.3f' % sklearn.metrics.v_measure_score(train_labels, kmeans_labels))

In [ ]:
# plot clustered images
# NOTE: the cluster lables are NOT associated with the orignal class labels
n_image_per_cluster = 15
n_cluster = 10
for cluster_label in np.arange(n_cluster):
    # get indices of the first n_image_per_cluster in this cluster
    itrains = np.where(kmeans_labels == cluster_label)[0][0:n_image_per_cluster]
    fig = plt.figure(dpi=100, figsize=(n_image_per_cluster * 2, n_cluster * 2))
    print('\n\nImages in cluster %d:' % cluster_label)
    for iplot, itrain in enumerate(itrains):
        subplot_image(train_images[itrain], string_labels[train_labels[itrain]], 
                      n_cluster, n_image_per_cluster, iplot)
    plt.show()

### The about results show that
* similar-looking images such as Shirts, Coats and Pullovers are not well distinguished
* Bags with and without handles are clustered into two subclasses 

### Show clustering in feature space

In [ ]:
# number of data to show
show_ndata = 500

# clusters to show
show_clusters = np.array([1, 2, 3])

# find the data subset
show_data_indices = np.where(np.isin(kmeans_labels, show_clusters))[0][0:show_ndata]

# normalise features to [0, 1]
train_images_encoded_norm = (train_images_encoded - train_images_encoded.min()) / \
                            (train_images_encoded.max() - train_images_encoded.min())
# for better visualisation
train_images_encoded_norm = np.power(train_images_encoded_norm, .5)

# plot
plt.figure(dpi=100, figsize=(15, 15))
iplot = 0
for feature_x in np.arange(bottleneck_all):
    for feature_y in np.arange(bottleneck_all):
        plt.subplot(bottleneck_all, bottleneck_all, iplot + 1)
        x = train_images_encoded_norm[show_data_indices, feature_x]
        y = train_images_encoded_norm[show_data_indices, feature_y]
        plt.scatter(x, y, c=kmeans_labels[show_data_indices], s=10, cmap=plt.cm.Paired)
        plt.xlim(-.05, .95)
        plt.ylim(-.05, .95)
        plt.xticks([])
        plt.yticks([])
        iplot += 1
        if feature_y == 0:
            plt.ylabel('Feature %d' % feature_x)
        if feature_x == 0:
            plt.gca().xaxis.set_label_position('top') 
            plt.xlabel('Feature %d' % feature_y)
plt.tight_layout()
plt.show()

---